In [1]:
import torch.nn as nn
import torch

# AvgPool через свертку

In [8]:
batch_size = 1
channels = 2
in_features = 3

kernel = in_features
stride = kernel
padding = 0


with torch.no_grad():

    # Генерация рандомного батча для проверки
    batch = torch.rand(batch_size, channels, in_features, in_features)

    print('Input data:')
    print(batch, '\n')

    print('Shape:')
    print(batch.shape, '\n')

    # Веса для свертки, которая выполняет роль пулинга
    weights = torch.full((channels, 1, kernel, kernel), 1 / kernel ** 2)
    
    print('Weights:')
    print(weights, '\n')

    print('Shape:')
    print(weights.shape, '\n')

    # Пулинг для сравнения со сверткой
    avg_pool = nn.AvgPool2d(kernel, stride = stride, padding = padding)

    # Свертка, выполняющая пулинг
    conv = nn.Conv2d(channels, channels, kernel_size = kernel, bias = False, 
                     groups = channels, stride = stride, padding = padding)
    
    conv.weight = nn.Parameter(weights)

# Сравнение результатов
print('Average pooling:')
print(avg_pool(batch), '\n')

print('Shape:')
print(avg_pool(batch).shape, '\n')

print('Сonvolution:')
print(conv(batch), '\n')

print('Shape:')
print(conv(batch).shape)

Input data:
tensor([[[[0.2411, 0.5594, 0.6807],
          [0.6958, 0.3491, 0.8149],
          [0.7596, 0.0656, 0.9756]],

         [[0.4597, 0.4995, 0.7872],
          [0.3004, 0.0459, 0.7954],
          [0.4192, 0.2110, 0.4955]]]]) 

Shape:
torch.Size([1, 2, 3, 3]) 

Weights:
tensor([[[[0.1111, 0.1111, 0.1111],
          [0.1111, 0.1111, 0.1111],
          [0.1111, 0.1111, 0.1111]]],


        [[[0.1111, 0.1111, 0.1111],
          [0.1111, 0.1111, 0.1111],
          [0.1111, 0.1111, 0.1111]]]]) 

Shape:
torch.Size([2, 1, 3, 3]) 

Average pooling:
tensor([[[[0.5713]],

         [[0.4460]]]]) 

Shape:
torch.Size([1, 2, 1, 1]) 

Сonvolution:
tensor([[[[0.5713]],

         [[0.4460]]]], grad_fn=<ConvolutionBackward0>) 

Shape:
torch.Size([1, 2, 1, 1])


# FC через свертку

In [11]:
#batch_size = 2
channels = 1

in_features = 2
out_features = 3

with torch.no_grad():

    #batch = torch.rand(batch_size, channels, in_features)

    # Генерация рандомного батча для проверки
    batch = torch.rand(channels, in_features)

    print('Input data:')
    print(batch, '\n')

    print('Shape:')
    print(batch.shape, '\n')

    # FC слой для сравнения со сверткой
    fc = nn.Linear(in_features, out_features, bias = False)

    print('FC weights:')
    print(fc.weight, '\n')

    # Свертка, выполняющая роль FC слоя
    conv = nn.Conv1d(channels, channels, kernel_size = out_features * in_features, stride = in_features, 
                     padding = in_features * (out_features - 1), groups = channels, bias = False)

    # Веса для свертки
    weights = fc.weight.flip(0)                                        # Отражение порядка следования весов, т.е.
                                                                       # веса будут перечисляться от нижнего нейрона к верхнему 

    weights = weights.flatten()                                        # Вытягивание матрицы весов в вектор
    weights = weights.repeat(1, channels)                              # Дублировать вектор весов для каждого канала
    weights = weights.reshape(channels, 1, out_features * in_features) # Перевод в матрицу весов для каждого канала

    conv.weight = nn.Parameter(weights)

# Сравнение результатов
print('FC:')
print(fc(batch), '\n')

print('Сonvolution:')
print(conv(batch), '\n')

Input data:
tensor([[0.5843, 0.3435]]) 

Shape:
torch.Size([1, 2]) 

FC weights:
Parameter containing:
tensor([[-0.2205,  0.2830],
        [ 0.5544,  0.5140],
        [-0.0907,  0.0859]], requires_grad=True) 

FC:
tensor([[-0.0316,  0.5005, -0.0235]], grad_fn=<MmBackward0>) 

Сonvolution:
tensor([[-0.0316,  0.5005, -0.0235]], grad_fn=<SqueezeBackward1>) 

